# Descover trade tendency, identify key trading partners, analyze specific product categories of Georgia in 2022
*** ***

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import squarify
import plotly.express as px
import plotly.graph_objs as go
import calendar

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', None)
pd.set_option('display.expand_frame_repr', True)

In [3]:
# plot variables.
label_size=15
title_size=22
fmt = ticker.EngFormatter(places=0)

## Data collection

In [4]:
jan_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_01.csv', dtype={"საქონლის კოდი":"object"})
feb_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_02.csv', dtype={"საქონლის კოდი":"object"})
mar_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_03.csv', dtype={"საქონლის კოდი":"object"})
apr_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_04.csv', dtype={"საქონლის კოდი":"object"})
may_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_05.csv', dtype={"საქონლის კოდი":"object"})
jun_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_06.csv', dtype={"საქონლის კოდი":"object"})
jul_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_07.csv', dtype={"საქონლის კოდი":"object"})
aug_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_08.csv', dtype={"საქონლის კოდი":"object"})
sep_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_09.csv', dtype={"საქონლის კოდი":"object"})
oct_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_10.csv', dtype={"საქონლის კოდი":"object"})
nov_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_11.csv', dtype={"საქონლის კოდი":"object"})
dec_22 = pd.read_csv('/kaggle/input/georgia-trade-2022/2022_12.csv', dtype={"საქონლის კოდი":"object"})


In [5]:
# Dataframe for with english and georgian names of countries.
countries = pd.read_csv('/kaggle/input/others/countries.csv')

# HS code names.
codes = pd.read_csv('/kaggle/input/others/codes.csv', dtype={"Category":"object"})
countries.head()

,Partner,Country
0,ავსტრალია,Australia
1,ავსტრია,Austria
2,ავღანეთი,Afghanistan
3,აზერბაიჯანი,Azerbaijan
4,ალბანეთი,Albania


In [6]:
codes.head()

,Category,Name
0,2603,Copper ores and concentrates
1,3102,"Mineral or chemical fertilisers, nitrogenous"
2,7202,Ferro-alloys
3,2201,Waters (natural or artificial)
4,2202,Mineral Waters


In [7]:
jan_22.head()

,საქონლის კოდი,რეჟიმი,რაოდენობა,რაოდენობა კგ,ღირებულება დოლარი,პარტნიორი ქვეყანა,გრაფა 29
0,01022110000,იმპორტი,33.00,18000.00,67828.70,ესტონეთი,69101
1,01022110000,იმპორტი,32.00,18500.00,88212.30,უნგრეთი,69101
2,01022110000,იმპორტი,32.00,18500.00,66274.17,უნგრეთი,69604
3,01022999000,იმპორტი,4.00,1825.00,5800.00,უკრაინა,69501
4,01039219000,იმპორტი,6728.00,797590.00,1203615.85,რუსეთი,69101


## Data Cleaning and transformation
### Handling Missing Values, Standardizing Formats, and Removing Anomalies

In [8]:
# Create list of above dataframes.
data = [jan_22, feb_22,mar_22,apr_22,may_22,jun_22,jul_22,aug_22,sep_22,oct_22,nov_22,dec_22]
months = ['January', 'February', 'March','April','May', 'June', 'July', 'August','September','October','November','December']

# Change column names for each dataframe.
for df, month in zip(data, months):
    df.rename(columns={'საქონლის კოდი':'Code',
                           'რეჟიმი':'Type',
                           'რაოდენობა':'Quantity',
                           'რაოდენობა კგ':'Quantity_kg',
                           'ღირებულება დოლარი':'Cost',
                           'პარტნიორი ქვეყანა':'Partner_country'
                          }, inplace=True)
#     remove unneccesary column.
    df.drop(columns='გრაფა 29', inplace=True)
#     add month column.
    df['Month']=month
    
# Combine dataframes for each month into a single one. 
df_2022 = pd.concat(data,axis=0, sort=False, ignore_index=True)
df_2022.head()

,Code,Type,Quantity,Quantity_kg,Cost,Partner_country,Month
0,01022110000,იმპორტი,33.00,18000.00,67828.70,ესტონეთი,January
1,01022110000,იმპორტი,32.00,18500.00,88212.30,უნგრეთი,January
2,01022110000,იმპორტი,32.00,18500.00,66274.17,უნგრეთი,January
3,01022999000,იმპორტი,4.00,1825.00,5800.00,უკრაინა,January
4,01039219000,იმპორტი,6728.00,797590.00,1203615.85,რუსეთი,January


In [9]:
# Exemaine dataframes's values types.
print('DataFrame Info -->', df_2022.info(),'\n')
print('Counte Unique Values--> \n', df_2022.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415112 entries, 0 to 415111
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Code             415112 non-null  object 
 1   Type             415112 non-null  object 
 2   Quantity         415112 non-null  float64
 3   Quantity_kg      415112 non-null  float64
 4   Cost             415112 non-null  float64
 5   Partner_country  415112 non-null  object 
 6   Month            415112 non-null  object 
dtypes: float64(3), object(4)
memory usage: 22.2+ MB
DataFrame Info --> None 

Counte Unique Values--> 
 Code                 7511
Type                    2
Quantity            96877
Quantity_kg        114793
Cost               278063
Partner_country       153
Month                  12
dtype: int64


In [10]:
# Correct country name.
df_2022['Partner_country'].replace('საუდების არაბეთი','საუდის არაბეთი',inplace=True)

# Change mode values with english related words.
df_2022.replace({'Type':{'იმპორტი':'Import', 'ექსპორტი':'Export'}}, inplace=True)

In [11]:
# Merge df_2022 and countries dataframe to import countries english name column.
df_2022 = pd.merge(df_2022, countries,left_on='Partner_country', right_on='Partner', how='left', sort=False)

# Create category column.
df_2022['Category'] = df_2022['Code'].str[:4]


# Merge df_2022 dataframe to codes dataframe to import categories' names.
df_2022 = pd.merge(df_2022, codes, on='Category', how='left', sort=False).fillna('')

# Show first 5 rows.
df_2022.head()

,Code,Type,Quantity,Quantity_kg,Cost,Partner_country,Month,Partner,Country,Category,Name
0,01022110000,Import,33.00,18000.00,67828.70,ესტონეთი,January,ესტონეთი,Estonia,0102,Live bovine animals
1,01022110000,Import,32.00,18500.00,88212.30,უნგრეთი,January,უნგრეთი,Hungary,0102,Live bovine animals
2,01022110000,Import,32.00,18500.00,66274.17,უნგრეთი,January,უნგრეთი,Hungary,0102,Live bovine animals
3,01022999000,Import,4.00,1825.00,5800.00,უკრაინა,January,უკრაინა,Ukraine,0102,Live bovine animals
4,01039219000,Import,6728.00,797590.00,1203615.85,რუსეთი,January,რუსეთი,russia,0103,


In [12]:
# Drop redundant columns
df_2022.drop(columns=['Partner_country', 'Partner'], inplace=True)

# Rearrange data columns
data_22 = df_2022[['Category','Code','Name','Type','Quantity','Quantity_kg','Cost','Country','Month']]
data_22.tail()

,Category,Code,Name,Type,Quantity,Quantity_kg,Cost,Country,Month
415174,9619,96190081000,,Export,2041.50,2041.50,7504.00,russia,December
415175,9620,96200010000,,Export,14.20,14.20,360.57,Iran,December
415176,9701,97011000000,,Export,5.00,8.00,35.00,Hungary,December
415177,9905,99050000001,,Export,6400.00,6400.00,71834.27,russia,December
415178,9905,99050000002,,Export,1195.00,1195.00,1525.00,France,December


In [13]:
# exclude non-declared trade
# df_2022 = df_2022[~(df_2022['country'].isin(['unknown','Not specified']))]

In [14]:
data_22.describe()

,Quantity,Quantity_kg,Cost
count,415179.00,415179.00,415179.00
mean,24924.70,29960.99,33697.42
std,468034.69,1100256.72,514464.94
min,0.00,0.00,0.00
25%,8.40,9.72,243.16
50%,81.21,91.88,1497.60
75%,985.00,1120.00,9226.72
max,96096057.00,287503687.49,104466576.04


## Total Trade Volume

Calculate merchandise trade balance. Calculate Total Export and Import of 2022

In [15]:
# total import & total export, quantities and cost.
total_export = data_22[data_22['Type']=='Export']['Cost'].sum()
total_import = data_22[data_22['Type']=='Import']['Cost'].sum()

# Calculate balance of trade (BOT) = Value of Exports − Value of Imports
trade_balance = total_export - total_import
print(f'{trade_balance:,.2f}')

-6,215,481,879.73


**Georgia has a trade deficit (negative trade balance).**

<br>

> "A negative balance of trade means that currency flows outwards to pay for exports, indicating that the country may be overly reliant on foreign goods." - __[investopedia](https://www.investopedia.com/terms/b/bot.asp)__


## Top Trade Partners
### Identify Georgia's Key Trade Partners

In [16]:
# Separate data into exports and imports
imports = data_22[data_22['Type']=='Import']
exports = data_22[data_22['Type']=='Export']

# Analyze distribution of trade costs for exports
exports['Cost'].describe()

count       36543.00
mean       106381.22
std       1379346.70
min             0.01
25%           343.08
50%          2880.64
75%         19351.75
max     104466576.04
Name: Cost, dtype: float64

In [17]:
# Create method that calculates share of total (export/import).
def getPct(arr, key):
    return arr[key].apply(lambda x:(x/sum(arr[key]))*100).round(2)

In [18]:
# Groupe countries by export and import and sort in descending order based on the “cost” column.
exporters = exports.groupby(['Country']).sum(numeric_only=True).fillna(0).sort_values('Cost', ascending=False).reset_index()
importers = imports.groupby(['Country'], as_index=False).sum(numeric_only=True).fillna(0).sort_values('Cost', ascending=False)

# Add "share" column to them
exporters['Share'] = getPct(exporters,'Cost')
importers['Share'] = getPct(importers,'Cost')

In [19]:
# Create Top 10 countries dataframes by export/import with highest trade volumes 
top_export_partners = exporters.head(10)
top_import_partners = importers.head(10)

In [20]:
top_export_partners

,Country,Quantity,Quantity_kg,Cost,Share
0,China,293815696.13,296695119.05,703502830.03,18.10
1,russia,399710201.94,475532427.00,551726786.65,14.19
2,Bulgaria,175046339.74,178295640.93,423065279.28,10.88
3,Türkiye,183753749.29,163846803.79,366507208.60,9.43
4,United States,113125504.62,115916468.88,263568466.75,6.78
5,Armenia,293291110.39,281679877.87,212639327.81,5.47
6,Azerbaijan,193675484.83,143961807.33,166878539.22,4.29
7,Peru,86284479.00,248644479.00,153820385.11,3.96
8,Ukraine,58796845.51,86725860.63,94889520.31,2.44
9,Switzerland,20936851.44,18396500.85,90060875.91,2.32


The top exporting countries and their respective shares of the total exports are as follows:

China accounts for the highest share with **18.10%** of the total exports.
russia follows closely with a significant contribution of **14.19%**.
Bulgaria is also notable, representing **10.88%** of the overall export share.

In [21]:
top_import_partners

,Country,Quantity,Quantity_kg,Cost,Share
124,Türkiye,1762922810.68,1505906271.18,2060897655.54,20.40
136,russia,2025367701.74,2320060621.93,1779976789.05,17.62
23,China,506306286.39,289417492.89,1062562666.38,10.52
7,Azerbaijan,1293607570.02,3269099203.45,620265944.74,6.14
44,Germany,120033121.83,73193238.30,388520704.98,3.85
100,Romania,112638077.37,252910441.54,273889048.34,2.71
56,Italy,35959172.52,42197987.89,255213153.34,2.53
125,UA Emirates,25692675.66,15362972.99,245261166.38,2.43
126,Ukraine,224887486.17,118835726.48,227858234.82,2.26
52,Iran,258315147.58,367043417.97,209450855.18,2.07


In [22]:
# Five countries by export with lowest trade volumes 
exporters.nsmallest(5, 'Cost')

,Country,Quantity,Quantity_kg,Cost,Share
129,Malawi,288.00,288.00,665.00,0.00
128,Gambia,211.00,235.00,870.00,0.00
127,Bangladesh,312.00,312.00,1500.00,0.00
126,Djibouti,71.60,69.40,1596.00,0.00
125,Seychelles,180.00,41.00,2102.74,0.00


In [23]:
# Five countries by import with lowest trade volumes 
importers.nsmallest(5, 'Cost')

,Country,Quantity,Quantity_kg,Cost,Share
121,Togo,16.00,16.00,173.93,0.00
134,Zambia,0.90,0.90,252.51,0.00
135,Zimbabwe,4.50,4.50,320.49,0.00
25,Congo,66.00,66.00,1200.00,0.00
62,Kuwait,434.60,434.60,3762.04,0.00


In [24]:
# Create a bar plot
plt_df = top_export_partners

fig = px.bar(plt_df, x='Cost', y='Country',
             hover_name='Country',
             hover_data={'Cost': ':,.2f', 'Share': ':.2f%'},      
             labels={'Cost': 'Total Export (USD)', 'Share': 'Share of Total (%)'},
             title='TOP COUNTRIES BY EXPORT IN 2022',
             text=plt_df['Cost'],
             template='plotly_dark'
            )
# Create the bar plot
fig.update_traces(texttemplate='$%{text:,.4s}', textposition='outside', marker_color= 'red')

# Add the 'share' values as text on the bars
fig.add_trace(
    go.Scatter(
        y=plt_df['Country'],
        x=plt_df['Share'],
        mode='text',
        text=plt_df['Share'],
        textposition='middle right',
        textfont=dict(size=12),
        texttemplate='%{text:.2f}%',
        showlegend=False,
        
    )
)

# Customize the layout
fig.update_layout(
    xaxis=dict(title='Cost'),
    yaxis=dict(title='Country'),
    hovermode='x unified'
)

# Show the plot
fig.show()

In [25]:
# Create a bar plot
plt_df = top_import_partners

fig = px.bar(plt_df, x='Cost', y='Country',
             hover_name='Country',
             hover_data={'Cost': ':,.2f', 'Share': ':.2f%'},      
             labels={'Cost': 'Total Export (USD)', 'Share': 'Share of Total (%)'},
             title='TOP COUNTRIES BY IMPORT IN 2022',
             text=plt_df['Cost'], template='plotly_dark')

# Shorten USD number.
fig.update_traces(texttemplate='$%{text:,.4s}',textposition='outside',marker_color= 'green')

# Add the 'share' values as text on the bars
fig.add_trace(
    go.Scatter(
        y=plt_df['Country'],
        x=plt_df['Share'],
        mode='text',
        text=plt_df['Share'],
        textfont=dict(size=12),
        textposition='middle right',
        texttemplate='%{text:.2f}%',
        showlegend=False,
        
    )
)


# Customize the layout
fig.update_layout(
    hovermode='x unified'
)
# Show the plot
fig.show();

#### Determine Trade Balance:

1. Determine the trade balance (BOT) for each country.
2. Identify countries with trade surpluses or deficits.

In [26]:
country_trade = pd.pivot_table(data_22,
               index=['Country'],
               columns=['Type'],
               values='Cost',
               aggfunc='sum',
               sort=False,
               fill_value=0
              ).reset_index()
# Create trade balance column for each country.
country_trade['Trade_balance'] = country_trade['Export'].sub(country_trade['Import'])

In [27]:
country_trade.head()

Type,Country,Import,Export,Trade_balance
0,Estonia,10382444.82,4642471.40,-5739973.42
1,Hungary,53444163.83,2816269.03,-50627894.80
2,Ukraine,227858234.82,94889520.31,-132968714.51
3,russia,1779976789.05,551726786.65,-1228250002.40
4,Armenia,106555889.38,212639327.81,106083438.43


In [28]:
# Filter country_trade dataframe.
df = country_trade[(country_trade['Export']>31516579) & (country_trade['Import']>59817728)]

# Create plot.
fig = px.bar(df,
             x='Country',
             y=['Export', 'Import', 'Trade_balance'],         
             title='Trade Balance (BOT) for Each Country',
             labels={'value': 'Total Trade Value (in USD)', 'variable': 'Trade Component'},
             hover_data={'value': ':,.2f'},
             template='plotly_dark',
             color_discrete_map={'Export': 'green', 'Import': 'red', 'Trade_balance': 'blue'},
             height=500
            )

# Customize the layout.
fig.update_layout(barmode='group',
                  showlegend=True,
                  yaxis=dict(title='Cost', tickformat=',.0s')
                 )

# Show the plot.
fig.show()

#### 10 countries with high trade deficits

In [29]:
country_trade.nsmallest(10,'Trade_balance')

Type,Country,Import,Export,Trade_balance
5,Türkiye,2060897655.54,366507208.60,-1694390446.94
3,russia,1779976789.05,551726786.65,-1228250002.40
39,Azerbaijan,620265944.74,166878539.22,-453387405.52
21,China,1062562666.38,703502830.03,-359059836.35
15,Germany,388520704.98,62430543.10,-326090161.88
46,Romania,273889048.34,17854509.53,-256034538.81
36,UA Emirates,245261166.38,26042634.52,-219218531.86
25,Italy,255213153.34,60189597.79,-195023555.55
38,Iran,209450855.18,15327203.77,-194123651.41
76,Turkmenistan,157360105.16,6320741.79,-151039363.37


#### Top 10 countries with high trade surpluses

In [30]:
country_trade.nlargest(10,'Trade_balance')

Type,Country,Import,Export,Trade_balance
45,Bulgaria,190983526.85,423065279.28,232081752.43
9,United States,91898123.95,263568466.75,171670342.80
116,Peru,303266.86,153820385.11,153517118.25
4,Armenia,106555889.38,212639327.81,106083438.43
60,Kazakhstan,36600097.60,73078882.57,36478784.97
59,Georgia,4873512.14,32607277.95,27733765.81
123,Gana,18589.17,20961982.20,20943393.03
80,Iraq,17133224.68,34801579.28,17668354.60
98,Saudi Arabia,4897172.80,19524591.61,14627418.81
139,Panama,8945.01,11222939.00,11213993.99


In [31]:
def sum_grouped_df(df,group_keys,col):
    return df.groupby(group_keys).sum(col).sort_values(col,ascending=False).reset_index()

In [32]:
# Filter dataframe by import and groupe it by categories
imp_cat_by_country = sum_grouped_df(imports,['Country','Category', 'Name'],'Cost')
exp_cat_by_country = sum_grouped_df(exports,['Country','Category', 'Name'],'Cost')

### Top Trade Partners' Top 10 Categories
#### Top Categories with Key Trade Partners

In [33]:
top_importers_top_cat = imp_cat_by_country.groupby('Country').head(10)
top_exporters_top_cat = exp_cat_by_country.groupby('Country').head(10)

In [34]:
# Group by 'Country' and 'Category' and sum the 'Quantity'
df = sum_grouped_df(top_importers_top_cat, ['Country', 'Name'],'Cost')

# Create a nested treemap using plotly.express
fig = px.treemap(df, path=['Country', 'Name'], values='Cost',
                 title='MAIN COUNTRIES BY IMPORT AND THEIR TOP IMPORTED PRODUCT CATEGORIES IN 2022',
                 width=900, height=700, template='plotly_dark')

# Show the treemap
fig.show()

In [35]:
# Group by 'Country' and 'Category' and sum the 'Quantity'
df = sum_grouped_df(top_exporters_top_cat,['Country', 'Name'],'Cost')

# Create a nested treemap using plotly.express
fig = px.treemap(df, path=['Country', 'Name'], values='Cost',
                 title='MAIN COUNTRIES BY EXPORT AND THEIR TOP EXPORTED PRODUCT CATEGORIES IN 2022',
                 width=900, height=700,template='plotly_dark'
                )

# Show the treemap
fig.show()

## Top Export-Import Product Categories
### Most imported product categories.

In [36]:
imported_categories = sum_grouped_df(imports, ['Category', 'Name'], 'Cost')
imported_categories.head(10)

,Category,Name,Quantity,Quantity_kg,Cost
0,2710,Petroleum oils,719765688.88,1303764653.99,1299100437.44
1,2711,Petroleum gases and other gaseous hydrocarbons,45594692.40,2166991446.46,403262607.67
2,3004,Medicaments,10775547.77,10775547.77,345664014.86
3,8517,Telephone sets and others,1830797.65,921186.45,313412582.27
4,8471,Automatic data-processing machines,854971.46,823037.74,134489940.44
5,1001,Wheat and meslin,368127182.54,368127182.54,129570858.08
6,7214,Other bars and rods of iron or non-alloy steel,167496196.98,167496196.98,118718009.17
7,1701,Cane or beet sugar and chemically pure sucrose...,151681554.11,151681554.11,89388508.63
8,2402,"Cigars, cheroots, etc.",252115734.25,5260633.91,87813662.68
9,8429,"Self-propelled bulldozers, excavators",1362.00,18039601.10,86155980.34


### Most exported product categories.

In [37]:
exported_categories = sum_grouped_df(exports, ['Category', 'Name'], 'Cost')
exported_categories.head(10)

,Category,Name,Quantity,Quantity_kg,Cost
0,2603,Copper ores and concentrates,416384564.00,416384564.00,978999019.44
1,7202,Ferro-alloys,228718290.50,228718290.50,453268931.72
2,3102,"Mineral or chemical fertilisers, nitrogenous",158226801.18,462215379.50,281420333.84
3,2204,Wine,88740140.90,148569434.81,251859494.35
4,2616,Precious-metal ores and concentrates,53935524.00,53935524.00,137293260.37
5,2208,Undenatured ethyl alcohol,20550796.22,53808629.92,124019715.58
6,2201,Waters (natural or artificial),269702785.34,291170463.79,111471253.14
7,0802,Nuts,23802664.10,23802664.10,99116780.66
8,2716,Electrical energy,11091651.91,543.00,93227141.88
9,2202,Mineral Waters,147963908.00,157170771.78,82335890.46


#### Identify categories that contribute significantly to a country's trade balance.

In [38]:
grouped_categories = pd.pivot_table(data_22, index=['Category', 'Name'], columns='Type', values = 'Cost', aggfunc='sum', sort=False).reset_index()
grouped_categories['Trade_balance'] = grouped_categories['Export'] - grouped_categories['Import']

In [39]:
# Calculate trade balance
main_contributors = grouped_categories.sort_values('Trade_balance', ascending=False).reset_index(drop=True)
main_contributors.head(10)

Type,Category,Name,Import,Export,Trade_balance
0,2603,Copper ores and concentrates,1000.00,978999019.44,978998019.44
1,7202,Ferro-alloys,8941065.22,453268931.72,444327866.50
2,3102,"Mineral or chemical fertilisers, nitrogenous",9255915.70,281420333.84,272164418.14
3,2204,Wine,4096556.29,251859494.35,247762938.06
4,2201,Waters (natural or artificial),120991.16,111471253.14,111350261.98
5,2208,Undenatured ethyl alcohol,27734468.13,124019715.58,96285247.45
6,0802,Nuts,7873115.97,99116780.66,91243664.69
7,7108,Gold,11806.39,81476704.96,81464898.57
8,2202,Mineral Waters,23931170.02,82335890.46,58404720.44
9,0102,Live bovine animals,3580857.10,59155284.30,55574427.20


## Monthly Dynamics of Georgia's Import and Export Activities

In [55]:
# Calculate total export and import by month.
monthly_report = pd.pivot_table(data_22, index='Month', columns='Type', values='Cost', aggfunc='sum', sort=False).reset_index()
# Calculate month-to-month percentage changes
monthly_report['Export_growth'] = monthly_report['Export'].pct_change().mul(100)
monthly_report['Import_growth'] = monthly_report['Import'].pct_change().mul(100)
# show data
monthly_report

Type,Month,Import,Export,Export_growth,Import_growth
0,January,614814418.02,245960982.93,NaN,NaN
1,February,746805194.42,337198608.38,37.09,21.47
2,March,699419128.40,346551187.54,2.77,-6.35
3,April,801197294.80,338544337.75,-2.31,14.55
4,May,830029451.44,380635880.70,12.43,3.60
5,June,839989013.12,282095425.14,-25.89,1.20
6,July,869559676.90,367653344.72,30.33,3.52
7,August,941637987.55,287980771.00,-21.67,8.29
8,September,882793325.62,349893511.65,21.50,-6.25
9,October,907164242.97,314899982.19,-10.00,2.76


In [56]:
# plt.figure(figsize=(14,8))
fig = px.line(monthly_report, x='Month',
              y=['Export', 'Import'],
              labels={'value': 'Amount'},
              hover_data={'value': ':,.2f'},
              color_discrete_sequence=['green', 'red'],
              title='Monthly Trends in Exports and Imports',
              template='plotly_dark',              
              markers=True,
             )

# Add annotations to the low and high points for each line
for x in ['Export', 'Import']:
    min_val = monthly_report[x].idxmin()
    max_val = monthly_report[x].idxmax()
    
    fig.add_annotation(
        x=monthly_report['Month'][min_val],
        y=monthly_report[x][min_val],
        text=f'${monthly_report[x][min_val]:,.0f}',
        showarrow=True,
        arrowhead=4,
        ax=0,
        ay=-40
    )
    
    fig.add_annotation(
        x=monthly_report['Month'][max_val],
        y=monthly_report[x][max_val],
        text=f'${monthly_report[x][max_val]:,.0f}',
        showarrow=True,
        arrowhead=4,
        ax=0,
        ay=-40,
#         font=dict(color='yellow')
    )
# Update layout for better visibility
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Amount', tickformat=',.0s'),
    hovermode='x unified',
    legend_title_text='Trade Type'
)
# Show the plot
fig.show()

Notably, Georgia experienced a significant increase in imports during December, reaching \\$1,039 billion, while the highest export value occurred in May, \\$318.64 milion.

In [57]:
# Plotting data
plt_data = monthly_report.dropna()
# Plotting Growth Rates
fig = px.line(plt_data, x='Month', y=['Export_growth', 'Import_growth'],
                    labels={'value': 'Growth Rate (%)', 'variable': 'Type'},
                    color_discrete_sequence=['green', 'red'],
                    template='plotly_dark',              
                    markers=True,
                    title='Monthly Growth Rates of Export and Import (2022)')

# Add annotations for growth rates
for trace in fig.data:
    for i, value in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=value,
            text=f'{value:.2f}%',
            showarrow=True,
            arrowhead=1,
            arrowcolor='red' if trace.name == 'Import Growth Rate' else 'yellow',
            ax=0,
            ay=-40
        )
fig.update_layout(xaxis_title='Month', yaxis_title='Growth Rate (%)')

# Show the plot
fig.show()

Analyzing the growth rates unveils interesting trends in the trade dynamics. For instance, February saw a notable increase in both export and import activities, with growth rates of **37.09%** and **21.47%**, respectively. On the other hand, June witnessed a substantial decline in export growth, registering at **-25.89%**.
Export growth experienced a spike in July and a notable increase in September.

**Key Observations:**

- June and August witnessed a substantial drop in export values, with a growth rates of **-25.89%**, **-21.67%**,respectively, reflecting a potential economic shift during that period.
- November and December showcased steady growth in both import and export activities, closing the year on a positive note.